In [ ]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import mean, desc
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)

sparkSession = SparkSession.builder.master("spark://winemap:7077") \
.config("spark.driver.extraClassPath","/opt/postgresql-42.1.4.jar") \
.getOrCreate()

In [ ]:
import psycopg2
conn = psycopg2.connect("host='172.17.0.4' port='5432' dbname='wineDb' user='username' password='password'")
cur = conn.cursor()
#make table
f = open(r'wineData.csv', 'r')
cur.copy_from(f, "wine_reviews", sep=',')
conn.commit()
f.close()

In [ ]:
import plotly.graph_objs as go

url = "jdbc:postgresql://172.17.0.4/wineDb?user=username&password=password"
df = (sparkSession.read.format("jdbc")
    .options(url=url, dbtable="wine_reviews")
    .load())

table = df.select('country','points').groupBy('country').agg(mean('points')).orderBy('avg(points)',ascending=False)
countryCols = table.select('country').show(5)
countries = list()
for country in countryCols:
    print(country[0])
    countries.append(str(country[0]))
pointCols = table.select('avg(points)')
points = list()
for point in pointCols:
    points.append(point[0])
data =  dict(type = 'choropleth',
        locationmode='country names',
        locations = countries,
        colorscale='Blues',
        z = points,
        colorbar = {'title': 'Average Rating'}
)
print(data)
layout = dict(geo = {'scope':'world'})
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)